Same usecase we are going to solve using regression  for finding salary of a customer

In [45]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder 
import pickle 

In [46]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [47]:
#Estimated salary is the output feature and remaing all are the input feature

In [48]:
#Preprocess the data
df.drop(labels=["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [49]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
#Encode the Categorical Variables
#Label encoder for gender
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df["Gender"])

In [51]:
#onehot ecnoding for geograhpy
onehot_enoder_geo=OneHotEncoder()
geo_encoded=onehot_enoder_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_enoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [52]:
#Combine onehot encoded columns with original data 
df=pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [53]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


Dividing The input and Output Features

In [54]:
X=df.drop("EstimatedSalary",axis=1)
y=df["EstimatedSalary"]

Train Test Split 

In [55]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Standardization 

In [56]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [57]:
#Saving all encoders and scaler in pickle formats in furthur use
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(onehot_enoder_geo,file)

with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

ANN Regression Problem Statement

In [58]:
import tensorflow as tf  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

Model Building 

In [59]:
model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),#HL1
    Dense(32,activation="relu"),#HL2
    Dense(1) #output layer
])

#compile the model
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])

In [60]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


setting Up logs 

In [61]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

#set up tensorboard
log_dir="regression/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorBoard_callback=TensorBoard(log_dir,histogram_freq=1)

In [62]:
#setting up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

Training the model 

In [63]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorBoard_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 100381.7109 - mae: 100381.7109 - val_loss: 98528.9609 - val_mae: 98528.9609
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99660.6953 - mae: 99660.6953 - val_loss: 97057.2734 - val_mae: 97057.2734
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 97093.5859 - mae: 97093.5859 - val_loss: 93287.8750 - val_mae: 93287.8750
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 92024.1016 - mae: 92024.1016 - val_loss: 86957.4453 - val_mae: 86957.4453
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 84586.6016 - mae: 84586.6016 - val_loss: 78747.6094 - val_mae: 78747.6094
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 75769.0000 - mae: 75769.0000 - val_loss: 70035.0234 - val_mae: 70035.0234
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 67066.7656 - mae: 67066.7656 

In [64]:
#Load tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [65]:
#%tensorboard --logdir regression/fit

Model evaluation on test data 

In [66]:
print(model.evaluate(X_test,y_test))

63/63 [==============================] - 0s 2ms/step - loss: 50277.2734 - mae: 50277.2734
[50277.2734375, 50277.2734375]


In [67]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

63/63 [==============================] - 0s 2ms/step - loss: 50277.2734 - mae: 50277.2734
Test Loss: 50277.2734375
Test MAE: 50277.2734375


saving the model using .h5 format 

In [68]:
model.save("regression_model.h5")

d:\Deep Learning Project Using ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
